## The Wonderful World of ML - Session 6: Data Wrangling in R - Part 1

### Why focus on data wrangling?

In the first of these sessions, we jumped right into a few of the basic ML algorithms: linear regresssion, logistic regression, LDA, QDA, and decision trees.  In this session we'll step back and look at data wrangling.  It's an important topic because the literature suggests [[1]](https://www.jstatsoft.org/article/view/v059i10/v59i10.pdf), it consumes roughly 80% of the time spent doing an analysis.

If it takes this much time just to get the data in a usable form, it makes sense to spend a little time in building some skills in this area.




### Connect to example database

We'll use the data from the dplyr tutorial given by Hadely Wickham here: [[2]](https://www.youtube.com/watch?v=8SGif63VW6E), [[3]](https://www.youtube.com/watch?v=Ue08LVuk790).  The database has been downloaded to the project so we can connect to it locally.  Here's a bit of code to do the connection and list out the tables in the database:

In [3]:
library(DBI)
library(RSQLite)
# connect to the sqlite file - currently in the notebooks dir of the project
# so we need to go up 2 levels to get to the data dir
con = dbConnect(SQLite(), dbname="../../data/2014_dplyr_flight_data/hflights.sqlite3")
# get a list of all tables
alltables = dbListTables(con)
alltables

[1] "airports"     "flights"      "planes"       "sqlite_stat1" "weather"

### All roads lead to Rome - Many ways to get the same result

A typical scenario which I've run into when I start some kind of analysis boils down to a series of choice that range from one of the two following extremes:

+ Write (what's often turns out to be) a complex SQL query to give you exactly what you need: both content and structure.
+ Write a simple SQL query that gets you the basic data content, but not in the right structure.  Then do a bunch of wrangling with dplyr to it into the right form.

This trade-off isn't apparent until you have do one or more joins to get the kind of dataframe you need to work on.  We'll show such an example, but for now, we'll keep it simple while we cover the basics.

### SQL Basics

TODO

In [7]:
query_string <- "SELECT * FROM flights"  # get all the records in the flights table
# Fetch all query results into a data frame:
flight_data <- dbGetQuery(con, query_string)
head(flight_data)
nrow(flight_data)

date,hour,minute,dep,arr,dep_delay,arr_delay,carrier,flight,dest,plane,cancelled,time,dist
14975,14,0,1400,1500,0,-10,AA,428,DFW,N576AA,0,40,224
14976,14,1,1401,1501,1,-9,AA,428,DFW,N557AA,0,45,224
14977,13,52,1352,1502,-8,-8,AA,428,DFW,N541AA,0,48,224
14978,14,3,1403,1513,3,3,AA,428,DFW,N403AA,0,39,224
14979,14,5,1405,1507,5,-3,AA,428,DFW,N492AA,0,44,224
14980,13,59,1359,1503,-1,-7,AA,428,DFW,N262AA,0,45,224


[1] 227496

Let's fix those dates.  This is the kind of stuff you'll typically run into when you start working with a new dataset.  After reading a little of the SQLite doc's on date types [[6]](https://www.sqlite.org/datatype3.html#date_and_time_datatype), we discover that there are no explicit "date" types.  Instead, dates are expressed as either TEXT, INTEGER, or REAL types.

So how do we find out what type

In [8]:
data_types_flights <- dbGetQuery(con, "Pragma table_info (flights)")
data_types_flights

cid,name,type,notnull,dflt_value,pk
0,date,REAL,0,NA,0
1,hour,INTEGER,0,NA,0
2,minute,INTEGER,0,NA,0
3,dep,INTEGER,0,NA,0
4,arr,INTEGER,0,NA,0
5,dep_delay,INTEGER,0,NA,0
6,arr_delay,INTEGER,0,NA,0
7,carrier,TEXT,0,NA,0
8,flight,INTEGER,0,NA,0
9,dest,TEXT,0,NA,0


In [13]:
query_string <- paste0("SELECT date(date, '6682 years', '38 days') as DATE, ",
                       "hour, minute, dep, arr, dep_delay, arr_delay, carrier, ",
                       "flight, dest, plane, cancelled, time, dist FROM flights")
flight_data <- dbGetQuery(con, query_string)
head(flight_data)
msg <- paste0("type before conversion: ", class(flight_data$DATE))
msg
suppressMessages(suppressWarnings(library(dplyr)))
flight_data <- mutate(flight_data, DATE = as.Date(DATE))
msg <- paste0("type AFTER conversion: ", class(flight_data$DATE))
msg

DATE,hour,minute,dep,arr,dep_delay,arr_delay,carrier,flight,dest,plane,cancelled,time,dist
2011-01-01,14,0,1400,1500,0,-10,AA,428,DFW,N576AA,0,40,224
2011-01-02,14,1,1401,1501,1,-9,AA,428,DFW,N557AA,0,45,224
2011-01-03,13,52,1352,1502,-8,-8,AA,428,DFW,N541AA,0,48,224
2011-01-04,14,3,1403,1513,3,3,AA,428,DFW,N403AA,0,39,224
2011-01-05,14,5,1405,1507,5,-3,AA,428,DFW,N492AA,0,44,224
2011-01-06,13,59,1359,1503,-1,-7,AA,428,DFW,N262AA,0,45,224


[1] "type before conversion: character"

[1] "type AFTER conversion: Date"

$$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\Yv}{\mathbf{Y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\betav}{\mathbf{\beta}}
\newcommand{\gv}{\mathbf{g}}
\newcommand{\Hv}{\mathbf{H}}
\newcommand{\dv}{\mathbf{d}}
\newcommand{\Vv}{\mathbf{V}}
\newcommand{\vv}{\mathbf{v}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\Sv}{\mathbf{S}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\Zv}{\mathbf{Z}}
\newcommand{\Norm}{\mathcal{N}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}
\newcommand{\dimensionbar}[1]{\underset{#1}{\operatorname{|}}}
\newcommand{\dimensionbar}[1]{\underset{#1}{\operatorname{|}}}
\newcommand{\grad}{\mathbf{\nabla}}
\newcommand{\ebx}[1]{e^{\wv_{#1}^T \xv_n}}
\newcommand{\eby}[1]{e^{y_{n,#1}}}
\newcommand{\Tiv}{\mathbf{Ti}}
\newcommand{\Fv}{\mathbf{F}}
\newcommand{\ones}[1]{\mathbf{1}_{#1}}
$$

### References

1. Wickham, Hadley, *Tidy Data*, Journal of Statistical Software, August 2014, Volume 59, Issue 10. [https://www.jstatsoft.org/article/view/v059i10/v59i10.pdf](https://www.jstatsoft.org/article/view/v059i10/v59i10.pdf)
2. Hadley Wickham's "dplyr" tutorial at useR 2014 (1/2) [https://www.youtube.com/watch?v=8SGif63VW6E](https://www.youtube.com/watch?v=8SGif63VW6E)
3. Hadley Wickham's "dplyr" tutorial at useR 2014 (2/2) [https://www.youtube.com/watch?v=Ue08LVuk790](https://www.youtube.com/watch?v=Ue08LVuk790)
4. Data sets for the 2014 dplyr tutorial, [https://www.dropbox.com/sh/i8qnluwmuieicxc/AAAgt9tIKoIm7WZKIyK25lh6a](https://www.dropbox.com/sh/i8qnluwmuieicxc/AAAgt9tIKoIm7WZKIyK25lh6a)
5. RSQLite documentation: [https://cran.r-project.org/web/packages/RSQLite/RSQLite.pdf](https://cran.r-project.org/web/packages/RSQLite/RSQLite.pdf)
6. SQLite documentation regarding date types: [https://www.sqlite.org/datatype3.html#date_and_time_datatype](https://www.sqlite.org/datatype3.html#date_and_time_datatype)
7. SQLite documentation regarding table structures (pragma): [http://www.sqlite.org/pragma.html#pragma_table_info](http://www.sqlite.org/pragma.html#pragma_table_info) 
8. 
